In [1]:
import json
with open('data/zero_rel_all.jsonl', 'r') as f:
    data = json.loads(next(f))
    # data = [json.loads(line) for line in f]

In [4]:
data.keys()

dict_keys(['ner', 'relations', 'tokenized_text'])

In [16]:
" ".join(data['tokenized_text'])

'SEOUL - South Korean and U.S. forces began their largest - ever live - fire exercises on Thursday , simulating a " full - scale attack " from North Korea , South Korea \'s defence ministry said . \n Some 2,500 troops from the South and the United States participated as the five - day drills kicked off in Pocheon near the border with the North , the ministry said . Multiple tanks , howitzers and fighter jets were also involved , it added . \n " The exercise demonstrated our military \'s capability and readiness to strongly respond to North Korea \'s nuclear and missile threats and to a full - scale attack , " the ministry said in a news release . \n Last week , North Korea \'s state media reported that leader Kim Jong Un had approved final preparations for the launch of the North \'s first military spy satellite . Analysts say the satellite will advancing the country \'s surveillance capability enabling it to strike targets more accurately in the event of a conflict . \n U.S. and South

In [14]:
data['relations'][:10]

[{'head': {'mention': 'SEOUL', 'position': ['0', '1'], 'type': 'GPE'},
  'tail': {'mention': 'South Korean', 'position': ['2', '4'], 'type': 'NORP'},
  'relation_text': 'location_of'},
 {'head': {'mention': 'SEOUL', 'position': ['0', '1'], 'type': 'GPE'},
  'tail': {'mention': 'U.S.', 'position': ['5', '6'], 'type': 'GPE'},
  'relation_text': 'exercises_with'},
 {'head': {'mention': 'SEOUL', 'position': ['0', '1'], 'type': 'GPE'},
  'tail': {'mention': 'Thursday', 'position': ['17', '18'], 'type': 'DATE'},
  'relation_text': 'exercise_started_on'},
 {'head': {'mention': 'SEOUL', 'position': ['0', '1'], 'type': 'GPE'},
  'tail': {'mention': 'North Korea', 'position': ['28', '30'], 'type': 'GPE'},
  'relation_text': 'near_location'},
 {'head': {'mention': 'SEOUL', 'position': ['0', '1'], 'type': 'GPE'},
  'tail': {'mention': "South Korea's",
   'position': ['31', '34'],
   'type': 'GPE'},
  'relation_text': 'location_of'},
 {'head': {'mention': 'SEOUL', 'position': ['0', '1'], 'type': 'G